In [1]:
from preprocess import drop_useless,incomplete_columns, min_miss_value_corr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_test=pd.read_csv('../../data/test_airbnb_berlin.xls')
data_train=pd.read_csv('../../data/train_airbnb_berlin.xls')

On visualise le nombre de lignes, colonnes ainsi que les différentes features dans le jeu de données.

In [2]:

rows = len(data_train.axes[0])
print(f'Nombre de lignes: {rows}\n')

cols = len(data_train.axes[1])
print(f'Nombre de colonnes: {cols}\n')

print(f"Colonnes:{data_train.columns.values}")

Nombre de lignes: 15692

Nombre de colonnes: 39

Colonnes:['Listing ID' 'Listing Name' 'Host ID' 'Host Name' 'Host Since'
 'Host Response Time' 'Host Response Rate' 'Is Superhost' 'neighbourhood'
 'Neighborhood Group' 'City' 'Postal Code' 'Country Code' 'Country'
 'Latitude' 'Longitude' 'Is Exact Location' 'Property Type' 'Room Type'
 'Accomodates' 'Bathrooms' 'Bedrooms' 'Beds' 'Square Feet'
 'Guests Included' 'Min Nights' 'Reviews' 'First Review' 'Last Review'
 'Overall Rating' 'Accuracy Rating' 'Cleanliness Rating' 'Checkin Rating'
 'Communication Rating' 'Location Rating' 'Value Rating'
 'Instant Bookable' 'Business Travel Ready' 'Price']


In [3]:
data_train.head(1)

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,Is Superhost,neighbourhood,Neighborhood Group,...,Overall Rating,Accuracy Rating,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Instant Bookable,Business Travel Ready,Price
0,19665213.0,*,156079597.0,Maximilian,2016-01-20,NaN,NaN,f,Prenzlauer Berg,Pankow,...,100.0,10.0,10.0,10.0,10.0,9.0,10.0,t,f,26.0


On enlève les features qui ne vont pas apporter d'informations.

In [4]:
data_train=drop_useless(data_train)

__Analyse du taux de missing par feature__

On liste les features pour lesquelles il manque des données.

In [5]:
listfeatures=incomplete_columns(data_train)

Square Feet => 15389 [98.07%]
Host Response Time => 7075 [45.09%]
Host Response Rate => 7075 [45.09%]
Checkin Rating => 2973 [18.95%]
Value Rating => 2972 [18.94%]
Accuracy Rating => 2971 [18.93%]
Cleanliness Rating => 2970 [18.93%]
Communication Rating => 2970 [18.93%]
Location Rating => 2971 [18.93%]
Overall Rating => 2962 [18.88%]
Is Superhost => 23 [0.15%]
Bathrooms => 14 [0.09%]
Price => 9 [0.06%]
Beds => 8 [0.05%]
Bedrooms => 5 [0.03%]


On remarque la présence de quelques astérix dans le dataset: nous choisissons de les remplacer par des NAN.

In [6]:
data_train = data_train.replace("*", pd.NA)

__Gestion des features à taux de "missing" extrême__

Lorsque l'on regarde les features où il manque moins de 30 données, on se rend compte que l'absence de ces données est MCAR. On se permet donc de supprimer les lignes où ces données sont manquantes. 
Il y a une exception sur la feature 'Is Superhost': si pour ces host il y a une note générale au dessus de 80, on les catégorises en Superhost, sinon, non. 

On commence par supprimer les features qui sont trop souvent manquantes (plus de 45%).

In [7]:
data_train=data_train.drop(columns=['Square Feet','Host Response Time','Host Response Rate'])

Pour les features presque toujours présentes (moins de 1% d'absence), on supprime les lignes concernées.

In [8]:
for col in ['Price','Bathrooms','Bedrooms','Beds']:
    data_train=data_train[data_train[col].notna()]

Si le host est noté, et ce au dessus de 80, il est supposé superhost, sinon il ne l'est pas

In [9]:
data_train['Is Superhost'][data_train['Is Superhost'].isna()] = np.where(data_train['Overall Rating'] > 80, 't', 'f')

__Recherche de corrélations entre les features présentant un taux de missing value "NaN" autour de 18%__

On va s'aider de la liste **listfeatures** définie plus tôt


In [10]:
data_train.columns

Index(['Is Superhost', 'neighbourhood', 'Neighborhood Group', 'Latitude',
       'Longitude', 'Property Type', 'Room Type', 'Accomodates', 'Bathrooms',
       'Bedrooms', 'Beds', 'Guests Included', 'Min Nights', 'Reviews',
       'Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating',
       'Value Rating', 'Instant Bookable', 'Business Travel Ready', 'Price'],
      dtype='object')

In [11]:
listfeatures_between_15_20 = [x for x in listfeatures.keys() if listfeatures[x]['pct']<20 and listfeatures[x]['pct']>15]

min_corr_pct=min_miss_value_corr(data_train,listfeatures_between_15_20)

print(f"Pourcentage minimal de corrélation entre deux features de la liste : {min_corr_pct}% \n")


Pourcentage minimal de corrélation entre deux features de la liste : 99.63% 



On peut donc raisonnablement affirmer que ces features sont de type MNAR : une listwise deletion n'est donc pas conseillée (et avec 18% d'instances concernées, ce n'est pas idéal)

__Enregistrement__

In [12]:
#fichier csv où l'on sans les ratings 
data_train_clean=data_train.drop(['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating',
       'Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating'], axis=1)
data_train_clean.to_csv('../../data/data_train_clean.csv',index=False)



#fichier csv où l'on garde les ratings

data_train_clean_ratings_included = data_train

for j in ['Overall Rating', 'Accuracy Rating', 'Cleanliness Rating','Checkin Rating', 'Communication Rating', 'Location Rating','Value Rating']:
       for i in data_train_clean_ratings_included[data_train_clean_ratings_included[j].isna()== True].index:
              data_train_clean_ratings_included=data_train_clean_ratings_included.drop(i, axis=0)

data_train_clean_ratings_included.to_csv('../../data/data_train_clean_ratings_included.csv',index=False)
